In [ ]:
!nvidia-smi

In [ ]:
import numpy as np

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
world.reset()

In [ ]:
import getpass
user = getpass.getuser()

In [ ]:
from omni.isaac.kit import SimulationApp

usd_path = f"omniverse://localhost/Users/{user}/test_allegro.usd"


simulation_app = SimulationApp({"headless": True, "open_usd": usd_path, "livesync_usd": usd_path})


In [ ]:
from omni.isaac.core import World
world = World()

In [ ]:
from hand_env import HandEnv

# env = HandEnv("/World/Hand/Bones/l_carpal_mid", "/World/Hand*/Bones/l_thumbSkeleton_grp/l_distalThumb_mid")
env = HandEnv("/World/allegro*/allegro_mount", "/World/AnchorXform")

In [ ]:
world.reset()

In [ ]:
env.start()
world.scene.add(env.robots)

In [ ]:
hand_pos, hand_rot = env.robots.get_world_poses()
hand_pos, hand_rot

In [ ]:
env.robots.get_joint_positions()

In [ ]:
for i, (path, limit) in enumerate(zip(env.robots._physics_view.dof_paths[0], env.robots.get_dof_limits()[0])):
    print(i, path, limit)

# World Pose

In [ ]:
target_pos, target_rot = (np.array([[0.2, 0., 0.]], dtype=np.float32), np.array([[1., 0., 0., 0.]], dtype=np.float32))

In [ ]:
env.robots.set_world_poses(target_pos, target_rot)

In [ ]:
world.step(render=True)

In [ ]:
world.reset()

# Stage one: move to the correct place

In [ ]:
from utils import get_mesh_bboxes
bboxes_list = get_mesh_bboxes("handle_6") 

In [ ]:
bboxes_list

In [ ]:
# get center and min x axis
min_x = bboxes_list[0][0][0] # 
center_list = [(e[1] + e[0]) / 2 for e in bboxes_list] # box center

In [ ]:
grasp_list = [[min_x - 0.1, c[1], c[2] - 0.12] for c in center_list]

In [ ]:
grasp_list

In [ ]:
graps_pos = np.array(grasp_list, dtype=np.float32)

base_rotation = [0.38268, 0, 0, 0.92388]
grasp_rot = np.array([base_rotation], dtype=np.float32)# XYZW
     

In [ ]:
graps_pos, grasp_rot

In [ ]:
world.reset()

In [ ]:
env.xforms.set_world_poses(graps_pos, grasp_rot)

In [ ]:
# env.robots.set_world_poses(graps_pos, grasp_rot)


In [ ]:
import time
time.sleep(10)

In [ ]:

for _ in range(500):
    world.step(render=True)

# Finger Pose

In [ ]:
tt = 120

In [ ]:
# pos = np.array([
#     [
#      0, 0, 0, np.pi/2, 
#      0,0,0,0,
#      0,0,0,0,
#      0,0,0,0,   
#     ],
# ])

# for i in range(tt):
#     env.robots.set_joint_position_targets(pos * i / tt) # 
#     world.step(render=True)
    
finger_pos = np.array([
    [
     0, 0, 0, np.pi/2 + np.pi/18, 
     np.pi/5, np.pi/5, np.pi/5, 0,
     np.pi/5, np.pi/5, np.pi/5, np.pi/6,
     np.pi/5, np.pi/5, np.pi/5, np.pi/6,
    ],
])

for i in range(tt):
    env.robots.set_joint_position_targets(finger_pos * i / tt) # 
    world.step(render=True)

# Stage last: pull

In [ ]:
graps_pos, grasp_rot

In [ ]:
# graps_pos[...,0] -= 0.05

In [ ]:
for i in range(200):
    graps_pos[...,0] -= 0.002
#   env.robots.set_world_poses(graps_pos, grasp_rot)
    env.xforms.set_world_poses(graps_pos, grasp_rot)
    env.robots.set_joint_position_targets(finger_pos)
    world.step(render=True)

In [ ]:
world.reset()